In [1]:
import firebase_admin
from firebase_admin import credentials, firestore, initialize_app, storage
import io
import os
from io import StringIO

from google.cloud import speech_v1p1beta1

In [2]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="john-depotube-personal-be14008820a6.json"

In [3]:
# Initialize Firestore DB
cred = credentials.Certificate('firebasekey.json')
default_app = initialize_app(cred)
db = firestore.client()
sampleCollection_ref = db.collection('sampleCollection')

In [4]:
doc_ref = sampleCollection_ref.document(u'Shea Jensen-9-17-19').get()

In [5]:
d = doc_ref.to_dict()
d['firmName']

'Hollins & Levy'

In [6]:
# Use custom bucket
bucket = storage.bucket('john-depotube-personal.appspot.com')

audioPath = "Shea Jensen-9-17-19-1of3_data/test_Long.wav" # path to local file being uploaded
audioBlob = bucket.blob("test_Long.wav") # create Blob object with name of file in firebase
audioBlob.upload_from_filename(audioPath) # do the actual upload

In [7]:
import time

blob_expiry = int(time.time() + (60 * 60 * 24 * 7)) # set for 7 days
signedURL = audioBlob.generate_signed_url(expiration=blob_expiry)

In [9]:
# local_file_path = "Shea Jensen-9-17-19-1of3_data/testseq3115to200.wav"
object_uri = 'gs://' + 'john-depotube-personal.appspot.com' + '/' + 'test_Long.wav'
client = speech_v1p1beta1.SpeechClient()

enable_word_confidence = True

# If enabled, each word in the first alternative of each result will be
# tagged with a speaker tag to identify the speaker.
enable_speaker_diarization = True
enable_wordtime_offsets = True
    
# The language of the supplied audio
language_code = "en-US"
config = {
        "enable_speaker_diarization": enable_speaker_diarization,
        "enable_word_confidence": enable_word_confidence,
        "language_code": language_code,
        #"enableWordTimeOffsets": enable_wordtime_offsets,
}
# with io.open(local_file_path, "rb") as f:
#    content = f.read()
audio = {"uri": object_uri}

    

In [10]:
operation = client.long_running_recognize(config, audio)

print(u"Waiting for operation to complete...")
# set timeout to (apparently) 4 hours
response = operation.result(timeout=14400)

Waiting for operation to complete...


In [11]:
# response = client.recognize(config, audio)
# The first result includes confidence levels per word
result = response.results[0]
# First alternative is the most probable result
alternative = result.alternatives[0]
print(u"Transcript: {}".format(alternative.transcript))
# Print the confidence level of each word
for word in alternative.words:
    print(u"Word: {}".format(word.word))
    print(u"Confidence: {}".format(word.confidence))
    print(u"Speaker tag: {}".format(word.speaker_tag))
    

Transcript: good afternoon we're going on the record at 2:24 p.m. on September 17th 2019 this is the video recorded deposition of a chance taken by counsel for plaintiffs in the matter of Karen Jones and Al versus prison
Word: good
Confidence: 0.9876290559768677
Speaker tag: 0
Word: afternoon
Confidence: 0.9876290559768677
Speaker tag: 0
Word: we're
Confidence: 0.9876290559768677
Speaker tag: 0
Word: going
Confidence: 0.9876290559768677
Speaker tag: 0
Word: on
Confidence: 0.9876290559768677
Speaker tag: 0
Word: the
Confidence: 0.9876290559768677
Speaker tag: 0
Word: record
Confidence: 0.9876290559768677
Speaker tag: 0
Word: at
Confidence: 0.9876290559768677
Speaker tag: 0
Word: 2:24
Confidence: 0.9876290559768677
Speaker tag: 0
Word: p.m.
Confidence: 0.9876290559768677
Speaker tag: 0
Word: on
Confidence: 0.9876290559768677
Speaker tag: 0
Word: September
Confidence: 0.9876290559768677
Speaker tag: 0
Word: 17th
Confidence: 0.9876290559768677
Speaker tag: 0
Word: 2019
Confidence: 0.811620

In [12]:
# response = client.recognize(config, audio)

In [13]:
# alternative.words
result

alternatives {
  transcript: "good afternoon we\'re going on the record at 2:24 p.m. on September 17th 2019 this is the video recorded deposition of a chance taken by counsel for plaintiffs in the matter of Karen Jones and Al versus prison"
  confidence: 0.9505285024642944
  words {
    start_time {
      seconds: 15
      nanos: 500000000
    }
    end_time {
      seconds: 17
      nanos: 800000000
    }
    word: "good"
    confidence: 0.9876290559768677
  }
  words {
    start_time {
      seconds: 17
      nanos: 800000000
    }
    end_time {
      seconds: 18
      nanos: 400000000
    }
    word: "afternoon"
    confidence: 0.9876290559768677
  }
  words {
    start_time {
      seconds: 18
      nanos: 400000000
    }
    end_time {
      seconds: 18
      nanos: 600000000
    }
    word: "we\'re"
    confidence: 0.9876290559768677
  }
  words {
    start_time {
      seconds: 18
      nanos: 600000000
    }
    end_time {
      seconds: 18
      nanos: 700000000
    }
    wor

In [14]:


soutput = StringIO()
for i in range(0, response.results.__len__()-1):
    alternative = response.results[i].alternatives[0]
    soutput.write(alternative.transcript + '\n')
    print (alternative.transcript)

# print soutput.getvalue()

transcriptionBlob = bucket.blob("testseq3115to200.txt") # name of file in firebase
transcriptionBlob.upload_from_string(soutput.getvalue()) # do the actual upload

good afternoon we're going on the record at 2:24 p.m. on September 17th 2019 this is the video recorded deposition of a chance taken by counsel for plaintiffs in the matter of Karen Jones and Al versus prison
Turks Dana Esquire at all case number BC 714
 429 this deposition is taking place at 21800 Oxnard Street and Suite 430 and Woodland Hills California
 Elkhound supposed to be please introduce yourselves and state from Europe to do the coffee a professional Corporation to the conference for Karen Jones House in Jones County Jones's present
 Byron Hollins representing defend dance along with Natalie lahiji and present is Shay Jensen my client
 would you say the sprayer from there for a leopard puss shefa Jensen name Nicholas
 have you use that name in a long time
 I'm sorry for the username in a while I mean to have you recently use that net last name necklace
 I don't know exactly how long but almost 20 years old
 including high school after you graduate I presume that you graduate 

In [15]:
transcriptionBlob = bucket.blob("test_Long.txt") # name of file in firebase
transcriptionBlob.upload_from_string(soutput.getvalue()) # do the actual upload

In [16]:
data = {

    u'firmName': u'Hollins & Levy',
    u'clientName': u'Shea Jensen',
    u'date': '09-17-19', # datetime.datetime.now(),
    u'videoURL': audioBlob.public_url,
    u'transcriptionURL': transcriptionBlob.public_url,
}

sampleCollection_ref.document(u'Shea Jensen-9-17-19').set(data)

update_time {
  seconds: 1597690638
  nanos: 995633000
}